In [5]:
import requests
from bs4 import BeautifulSoup
import re
def extractKyobo(ISBN: int) -> list:
    """

    새롭게 확보한 도서의 데이터를 교보문고에서 크롤링한다.
    목차, 도서소개, 추천사 등 가용한 텍스트 모두를 수집한다.
    해당 함수는 kyoboSaveMultiThread에서 사용된다.

    """
    kyoboUrl = f"http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode={ISBN}"
    kyoboHtml = requests.get(kyoboUrl)
    global kyoboSoup
    kyoboSoup = BeautifulSoup(kyoboHtml.content, "html.parser")
    try:
        bookTitle: str = kyoboSoup.h1.span.text

        contents = kyoboSoup.find_all(class_="box_detail_article")
        sortedItems = []
        for item in contents:
            if item.find(class_="content"):
                # 숨겨진 항목을 불러오는 조건식
                item = item.find_all(class_="content")[-1]

            result = re.sub("<.*?>|\W|[_*]", " ", str(item))
            sortedItems.append(result)

        itemList = [bookTitle, ISBN]
        itemList.extend(sortedItems)

    except:
        itemList = ["skip", ISBN]
        print("Skip :", ISBN)

    return itemList

extractKyobo(9791158393038)

Skip : 9791158393038


['skip', 9791158393038]

In [110]:
import requests
from bs4 import BeautifulSoup
import re
from itertools import chain

def _extract_title(soup):
    return soup.find(class_="prod_title").string

def _extract_intro(soup):
    book_intro_soup = soup.find_all("div", "info_text")[-1]
    book_intro_chunk = str(book_intro_soup).split("<br/>")
    book_intro_chunk_2d = list(
        map(lambda x: re.sub("<.*>", "", x).replace(".", ".##").split("##"), book_intro_chunk)
    )
    return list(chain(*book_intro_chunk_2d))


def _extract_toc(soup):
    book_toc_soup = soup.find(class_="book_contents_item")
    return str(book_toc_soup).split("<br/>")[1:-1]


def _extract_publisher(soup):
    publisher: str = soup.find(class_="book_publish_review")

    if publisher is not None:
        publisher = publisher.p.get_text().replace(".", ".##").split("##")
    else:
        publisher = []

    return publisher


def _clean_up_book_info(book_info):
    # 한국어 영어 제외하고 모두 제거
    book_info = (re.sub("[^A-Za-z\u3130-\u318F\uAC00-\uD7A3]", " ", i) for i in book_info)

    # 공백 하나로 줄이기 ex) '  ' -> ' '
    book_info = (re.sub(" +", " ", i).strip(" ") for i in book_info)

    book_info = filter(None, book_info)

    return list(book_info)



async def scrap_book_info(ISBN) :

    """
    book_title, book_toc, book_intro, publisher 추출하는 함수
    """

    url = f"http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode={ISBN}"

    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            html = await resp.content

    kyobo_soup =  BeautifulSoup(html, "html.parser")


    book_title: str = _extract_title(kyobo_soup)
    book_publisher: list = _extract_publisher(kyobo_soup)
    book_intro: list = _extract_intro(kyobo_soup)
    book_toc: list = _extract_toc(kyobo_soup)


    book_info = list(map(_clean_up_book_info, [book_toc, book_intro, book_publisher]))
    
    return [book_title] + book_info


In [46]:
from main import request_book_data
import asyncio
from datetime import date

lib_codes = [
    111003,
    111004,
    111005,
    111006,
    111007,
    111008,
    111009,
    111010,
    111022,
    111011,
    111012,
    111013,
    111014,
    111016,
    111030,
    111015,
    111018,
    111019,
    111020,
    111021,
]

chunk = 10
answer = []
for i in range(0, len(lib_codes), chunk):
    chunk_lib_codes = lib_codes[i : i + chunk]
    tasks = [request_book_data(i,start_date=date(2020, 1, 1)) for i in chunk_lib_codes]
    results = await asyncio.gather(*tasks)
    answer.extend(results)

ContentTypeError: 0, message='Attempt to decode JSON with unexpected mimetype: text/html', url=URL('http://data4library.kr/api/itemSrch?libCode=111010&startDt=2020-01-01&endDt=2023-03-09&authKey=7123eacb2744a02faca2508a82304c3bf154bf0b285da35c2faa2b8498b09872&pageSize=10000&format=json')

In [37]:
from itertools import chain

answer

lst = []
for i in chain(*results) :
    lst.append(i)


In [34]:
import pandas as pd 
pd.DataFrame(lst)

""


In [1]:
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import requests
lib_code = 111016
auth_key = "7123eacb2744a02faca2508a82304c3bf154bf0b285da35c2faa2b8498b09872"
end_date = datetime.now().date().replace(day=1) - timedelta(days=1)
# start_date = end_date - relativedelta(months=1)
start_date=date(2023, 1, 1)

libUrl = f"http://data4library.kr/api/itemSrch?libCode={lib_code}&startDt={start_date}&endDt={end_date}&authKey={auth_key}&pageSize=10000&format=json"

aaa = requests.get(libUrl,timeout=200)

aaaa = aaa.json()
all_books_info = map(lambda x: x["doc"], aaaa["response"]["docs"])

NameError: name 'date' is not defined